# Using dash
The scope of this notebook is to learn how to use dash

### Import of packages

In [42]:
#General
import pandas as pd
import numpy as np
#dash related
import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objs as go
import plotly
import plotly.plotly as py
plotly.__version__


'2.0.15'

### Import a datset to use for visualizations
We will be using the fifa 2017 dataset posted on Kaggle (https://www.kaggle.com/artimous/complete-fifa-2017-player-dataset-global)

In [34]:
df = pd.read_csv('Fifa_forDash.csv')
df.sort_values('Rating', inplace=True, ascending=False)
df = df.reset_index().drop('index', axis = 1)



### Map Height and Weight to numeric

In [35]:
df.Height = pd.to_numeric(df.Height.str.replace('cm', ''), errors='coerce')
df.Weight = pd.to_numeric(df.Weight.str.replace('kg', ''), errors='coerce')

In [36]:
df.head()

,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes
0,Cristiano Ronaldo,Portugal,LS,7.0,Real Madrid,LW,7.0,07/01/2009,2021.0,94,...,90,81,76,85,88,14,7,15,11,11
1,Lionel Messi,Argentina,RW,10.0,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,...,88,89,90,74,85,14,6,15,11,8
2,Neymar,Brazil,LW,10.0,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,...,77,79,84,81,83,15,9,15,9,11
3,Luis Suárez,Uruguay,LS,9.0,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,...,86,86,84,85,88,33,27,31,25,37
4,Manuel Neuer,Germany,GK,1.0,FC Bayern,GK,1.0,07/01/2011,2021.0,92,...,16,14,11,47,11,91,89,95,90,89


In [37]:
Nationalities = df.groupby('Nationality')['Name'].nunique().sort_values(ascending = False).index
Nat_counts = df.groupby('Nationality')['Name'].nunique().sort_values(ascending = False).values

## Dash with an interactive dropdown that defines the content of the second plot

In [38]:
#create a dash object
app = dash.Dash()

#data for first plot
data1 = [{'x': list(range(0,len(Nationalities))), 
         'y': Nat_counts, 
         'type': 'bar',
         'text': Nationalities,
         'name':'rating' }]

#data for second plot
df.sort_values('Rating', ascending=False)
selected = 'England'


#layout of the dash
app.layout = html.Div([
    #first plot (half width)
    html.Div([dcc.Graph(
              id='example-graph',
              figure={
                'data': data1,
                'layout': {'title': 'Number of players per nationality'}
               }
              )],
              style={'width': '48%', 'float': 'left', 'display': 'inline-block'}
            ),
    
    #second plot (half width)
    html.Div([dcc.Graph(id='weight_height'),], 
              style={'width': '48%', 'float': 'left', 'display': 'inline-block'}
            ),
    
    #dropdown with nationalities
    html.Div([
                dcc.Dropdown(
                    id='dropdown-nationality',
                    options=[{'label': i, 'value': i} for i in Nationalities],
                    value='England'
                ),
            #third plot 
                dcc.Graph(id='example-graph2')
                ##this plot is built in realtime using the output from a function
             ],
        style={'width': '48%', 'float': 'left', 'display': 'inline-block'} #specifies that is half width
    ),                                                                  #and should appear on the left

    #fourth plot 
    html.Div([dcc.Graph(id='example-graph3')],
              style={'width': '48%', 'float': 'right', 'display': 'inline-block'}
    )
]
)

###What is updated when the dash is run? FOR PLOT 2#########################################################
@app.callback(
    dash.dependencies.Output('weight_height', 'figure'),     ### This output is updated in realtime using 
                                                              ### the function 'update_graph'
    [dash.dependencies.Input('dropdown-nationality', 'value')])   ###The input from 'dropdown-nationality' is read

#######Function to update the plot as called by dash.dependencies.Output  #################################
def update_graph(selected):

    #Defines the data for the updated plot    
    #Other nationalities
    trace1 = go.Scatter(
                    x = df[df.Nationality != selected]['Height'],
                    y = df[df.Nationality != selected]['Weight'],
                    text = df.Name,
                    mode='markers',
                    marker=dict(
                            size='5',
                            color = '#7fbf7b'
                            )
                    )
    #Selected nationality
    trace2 = go.Scatter(
                    x = df[df.Nationality == selected]['Height'],
                    y = df[df.Nationality == selected]['Weight'],
                    text = df.Name,
                    mode='markers',
                    marker=dict(
                            size='7',
                            color = '#7b3294'
                            )
                    )
    #Defines the updated layout of the plot
    layout = go.Layout(title='Weight VS Height for ' + selected,
                       xaxis={'title': 'Height'},
                       yaxis={'title': 'Weight'},
                       hovermode='closest',
                       showlegend=False
                       )
    
    data = [trace1, trace2]
    #return a dictionary that defines the plot to be updated
    return {
        'data': data,
        'layout': layout
    }
###################################################################################################


                     
###What is updated when the dash is run? FOR PLOT 3#########################################################
@app.callback(
    dash.dependencies.Output('example-graph2', 'figure'),     ### This output is updated in realtime using 
                                                              ### the function 'update_graph'
     [dash.dependencies.Input('dropdown-nationality', 'value')])   ###The input from 'dropdown-nationality' is read

#######Function to update the plot as called by dash.dependencies.Output  ######################################
def update_graph(selected):
    df_subset = df[df['Nationality'] == selected]
    df_subset.reset_index(inplace=True)
    
    #Defines the data for the updated plot
    data2 = [{'x': df_subset.index, 
             'y': df_subset.Rating, 
             'type': 'scatter',
             'text': df_subset.Name,
             'name':'rating' }
            ]
    #Defines the updated layout of the plot
    layout = {'title': 'Player ratings for ' + selected,
                   'yaxis': dict(
                                autotick=False,
                                ticks='outside',
                                range=[40, 95],
                                dtick=5,
                                ticklen=8,
                                tickwidth=4,
                                tickcolor='#000'
                                )
                  }
    #return a dictionary that defines the plot to be updated
    return {
        'data': data2,
        'layout': layout
    }
###################################################################################################


###What is updated when the dash is run? FOR PLOT 4#########################################################
@app.callback(
    dash.dependencies.Output('example-graph3', 'figure'),     ### This output is updated in realtime using 
                                                              ### the function 'update_graph'
    [dash.dependencies.Input('dropdown-nationality', 'value')])   ###The input from 'dropdown-nationality' is read

#######Function to update the plot as called by dash.dependencies.Output  #################################
def update_graph(selected):
    df_subset = df[df['Nationality'] == selected]
    df_subset.groupby('Rating')['Name'].count()    
    grouped = df_subset.groupby('Rating')['Name'].count()

    #Defines the data for the updated plot
    data3 = [go.Bar(
            x = grouped.index,
            y = grouped.values,
            marker = dict(color = '#efedf5',
                          line=dict(color='#756bb1',
                                    width=1
                                   )
                         )
            )
            ]
    #Defines the updated layout of the plot
    layout = go.Layout(
            title='Player ratings distribution for ' + selected,
            xaxis={'title': 'Rating'},
            yaxis={'title': 'Count'},
            hovermode='closest',
        )
    
    #return a dictionary that defines the plot to be updated
    return {
        'data': data3,
        'layout': layout
    }
###################################################################################################

### Run the dash
The dash is the available at http://127.0.0.1:8050/

In [23]:
#To run the dash!    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


### Introduce country codes for plotting maps
Also, convert the name of the countries in order to match those 

In [39]:
#Import a df containing country names and country codes for mapping
country_code = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv').drop(
    'GDP (BILLIONS)', axis = 1)
df['Nationality'] = df['Nationality'].str.replace('China PR', 'China')
df['Nationality'] = df['Nationality'].str.replace('DR Congo', 'Congo, Democratic Republic of the')
df['Nationality'] = df['Nationality'].str.replace('FYR Macedonia', 'Macedonia')
df['Nationality'] = df['Nationality'].str.replace('Central African Rep.', 'Central African Republic')
df['Nationality'] = df['Nationality'].str.replace('Korea Republic', 'Korea, South')
df['Nationality'] = df['Nationality'].str.replace('São Tomé & Príncipe', 'Sao Tome and Principe')
df['Nationality'] = df['Nationality'].str.replace('Korea DPR', 'Korea, North')
df['Nationality'] = df['Nationality'].str.replace('St Kitts Nevis', 'Saint Kitts Nevis')
df['Nationality'] = df['Nationality'].str.replace('St Lucia', 'Saint Lucia')


df['Nationality'] = df['Nationality'].str.replace('Bosnia Herzegovina', 'Bosnia and Herzegovina')
df['Nationality'] = df['Nationality'].str.replace('St Lucia', 'Saint Lucia')
df['Nationality'] = df['Nationality'].str.replace('St Lucia', 'Saint Lucia')
df['Nationality'] = df['Nationality'].str.replace('St Lucia', 'Saint Lucia')




df = df.merge(country_code[['COUNTRY', 'CODE']], left_on='Nationality', right_on='COUNTRY', how = 'left')

In [40]:
Codes = df.groupby('CODE')['Name'].nunique().sort_values(ascending = False).index
Codes_counts = df.groupby('CODE')['Name'].nunique().sort_values(ascending = False).values

In [44]:
### FIXME: several countries are not shown
data = [ dict(
        type = 'choropleth',
        locations = Codes,
        z = Codes_counts,
        text = Codes,
        #colorscale = [[0,'rgb(180,180,180)'],[500,'rgb(180,180,180)'],[1000,'rgb(180,180,180)']],
        #autocolorscale = True,
        #reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
#            tickprefix = '$',
            title = 'Number Of Players'),
      ) ]

layout = dict(
    title = 'Number of players per nation',
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )